In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.transforms import ToTensor

train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [2]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [3]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    batch_size = batch_size,
    shuffle = False
)

In [4]:
import torch.nn as nn
class cnn(nn.Module): 
    def __init__(self): 
        super(cnn, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=13, kernel_size=2, padding=1) 
        self.conv2 = nn.Conv2d(13, 26, 2, padding=1)
        self.fc1 = nn.Linear(7*7*26, 128)
        self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        # convoultion # input (28 * 28 * 1)
        x = self.conv1(x) # (28 * 28 * 13)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2) # (14 * 14 * 13)
        x = self.conv2(x) # (14 * 14 * 26)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2) # (7 * 7 * 26)
        
        # fully connected
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x,dim=1)
        return x

In [5]:
model = cnn()

# loss and optimizer
learning_rate = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [6]:
class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

In [7]:
model

cnn(
  (conv1): Conv2d(1, 13, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(13, 26, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1274, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [8]:
# training
def test(model, criterion, optimizer, data):
    model.train() # training
    for X,y in data:
        y_pred = model(X)
        loss = criterion(y_pred,y)
        
        optimizer.zero_grad() # zero_grad(): 미분값 초기화
        loss.backward() # backpropagation
        optimizer.step() # update weights and biases

In [9]:
test(model, criterion, optimizer, test_loader)

In [10]:
def evaluation(model, criterion, optimizer, data):
    model.eval() #evaluation
    test_loss = 0
    accuracy = 0

    with torch.no_grad(): # no update
        for X, y in data:
            y_pred = model(X)
            test_loss += criterion(y_pred, y)
            
            prediction = y_pred.max(1)[1] # max indices
            corrects = (prediction == y)
            accuracy += corrects.sum().float() / float( y.size(0) )
    return test_loss, accuracy     

In [11]:
nums_epoch = 27
for epoch in range(nums_epoch):
    test(model, criterion, optimizer, test_loader)
    
    if(epoch%3==0):
        E, accu = evaluation(model, criterion, optimizer, test_loader)
        print(epoch, "loss=", E.item(), "accu=", accu.item())

0 loss= 169.92276000976562 accu= 76.260009765625
3 loss= 164.32135009765625 accu= 81.72999572753906
6 loss= 163.51710510253906 accu= 82.58998107910156
9 loss= 160.77920532226562 accu= 85.29000854492188
12 loss= 160.65261840820312 accu= 85.52999877929688
15 loss= 158.4467315673828 accu= 87.68000030517578
18 loss= 158.660888671875 accu= 87.45999145507812
21 loss= 157.03921508789062 accu= 89.06000518798828
24 loss= 157.03619384765625 accu= 89.0899887084961


## 감상문

In [ ]:
#딥러닝을 대하는 태도에 많은 영감을 받았다. 본인은 딥러닝을 학문으로 생각하여 기존의 방법을 정석이라 맹신하였고 
#정석에 몇 가지 기능이나 방법을 개선하는 것이 딥러닝의 발전의 능사라고 생각했기 때문이다.  
#허나 해당 사례를 보고 딥러닝을 학문이 아닌 공학 작품처럼 취급하는 것이 어쩌면 더 옳다는 느낌을 받았다.
#문제 해결에 어떤 도구가 필요하다면 이는 기존 도구들을 연계시켜 새로운 기능을 시키는게 아닌, 문제 해결에 필요한 도구를 만드는 것이
#올바른 방법이기 때문이다. 이런 생각은 50년간 쌓인 학계 노하우가 새로운 알고리즘으로 갱신되는 
#알파폴드 시리즈에 크게 뒤쳐졌단 사실에서 강하게 느껴졌었다 
#본인은 이 글의 화자가 마지막에 언급한 "나의 영역을 고수하는 것이 나를 그 안 가두어 놓는 시대" 에서 대표적인 잘못된 예라고 
#할 수 있을 정도로 본인을 가두어 놓는 사고 방식을 하고 있었으나 해당 사례를 통해 본인의 한계를 극복하는 것에 큰 도움이 되었다. 